## Install Required Libraries
We use the `psycopg2` library to connect to PostgreSQL and `pandas` for handling data.

In [1]:
!pip install psycopg2 pandas -q

### Connecting to PostgreSQL
We establish a connection to the PostgreSQL database using the provided credentials:
- **Username:** `postgres`
- **Password:** `your password`
- Default database: `your database name `


In [11]:
import psycopg2
import pandas as pd

# Database connection parameters
db_config = {
    "host": "localhost",  # Update this to your host, e.g., '127.0.0.1' or a remote IP
    "port": 5432,         # Default PostgreSQL port
    "database": "films_ds30",  # Update to the database you want to connect to
    "user": "postgres",
    "password": "P@ssw0rd"
}

# Connect to the PostgreSQL database
try:
    conn = psycopg2.connect(**db_config)
    print("Database connection established successfully.")
except Exception as e:
    print(f"Error connecting to the database: {e}")


Database connection established successfully.


# Explore Tables
Using the `information_schema.tables` system view, we list all tables in the database.


In [12]:
# Query to fetch all tables in the database
query = """
SELECT table_schema, table_name 
FROM information_schema.tables 
WHERE table_type = 'BASE TABLE' AND table_schema NOT IN ('information_schema', 'pg_catalog');
"""

try:
    tables = pd.read_sql(query, conn)
    print("Tables in the database:")
    print(tables)
except Exception as e:
    print(f"Error retrieving tables: {e}")


Tables in the database:
  table_schema table_name
0       public   accounts
1       public  employees
2       public      films
3       public     people
4       public    reviews
5       public      roles


C:\Users\Hassan\AppData\Local\Temp\ipykernel_3744\37775748.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tables = pd.read_sql(query, conn)


### Retrieve Column Names and Data Types
We will query the `information_schema.columns` view to get the column names and data types for each table in the database.


In [16]:
# Function to get column details for a specific table
def get_table_columns(table_name):
    query = f"""
    SELECT 
        column_name, 
        data_type 
    FROM 
        information_schema.columns 
    WHERE 
        table_name = '{table_name}';
    """
    try:
        # Execute the query and load the results into a DataFrame
        columns_info = pd.read_sql(query, conn)
        print(f"Columns for table '{table_name}':")
        print(columns_info)
        return columns_info
    except Exception as e:
        print(f"Error retrieving columns for table '{table_name}': {e}")
        return None

# Example: Get columns for the 'employees' table
table_name = "people"  # Replace with any table name from your list
employees_columns = get_table_columns(table_name)


Columns for table 'people':
  column_name          data_type
0          id            integer
1   birthdate               date
2   deathdate               date
3        name  character varying


C:\Users\Hassan\AppData\Local\Temp\ipykernel_3744\1661854942.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  columns_info = pd.read_sql(query, conn)


### Load Data into Pandas DataFrame
We select a specific table to load its contents into a Pandas DataFrame.

In [6]:
# Specify the table you want to load (update this with an actual table name from the output above)
table_name = "films"  # Replace with the desired table name

# Load the data into a Pandas DataFrame
try:
    data = pd.read_sql(f"SELECT * FROM {table_name};", conn)
    print(f"Data from {table_name}:")
    print(data.head())  # Display the first few rows of the table
except Exception as e:
    print(f"Error loading data from table '{table_name}': {e}")

Data from films:
   id                                             title  release_year  \
0   1  Intolerance: Love's Struggle Throughout the Ages        1916.0   
1   2                    Over the Hill to the Poorhouse        1920.0   
2   3                                    The Big Parade        1925.0   
3   4                                        Metropolis        1927.0   
4   5                                     Pandora's Box        1929.0   

   country  duration language certification      gross     budget  
0      USA     123.0     None     Not Rated        NaN   385907.0  
1      USA     110.0     None          None  3000000.0   100000.0  
2      USA     151.0     None     Not Rated        NaN   245000.0  
3  Germany     145.0   German     Not Rated    26435.0  6000000.0  
4  Germany     110.0   German     Not Rated     9950.0        NaN  


C:\Users\Hassan\AppData\Local\Temp\ipykernel_3744\1304803291.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(f"SELECT * FROM {table_name};", conn)


### Introduction to Window Functions
Window functions perform calculations across a set of rows related to the current row, producing a result for every row. Unlike aggregate functions (e.g., `SUM`, `AVG`), window functions do not group rows into a single output.

Key concept: The `OVER` clause defines the "window" over which the function operates. Without parameters, the `OVER` clause applies the function to the entire dataset.

Example: Using `ROW_NUMBER()` without additional parameters:
- Assigns a unique sequential number to each row based on the order in which they appear.


In [17]:
# ROW_NUMBER() applied over the entire dataset
query = """
SELECT 
    id, 
    title, 
    ROW_NUMBER() OVER () AS row_number
FROM films
LIMIT 10;
"""

try:
    result = pd.read_sql(query, conn)
    print("ROW_NUMBER() over all rows (no parameters):")
    print(result)
except Exception as e:
    print(f"Error executing query: {e}")


ROW_NUMBER() over all rows (no parameters):
   id                                             title  row_number
0   1  Intolerance: Love's Struggle Throughout the Ages           1
1   2                    Over the Hill to the Poorhouse           2
2   3                                    The Big Parade           3
3   4                                        Metropolis           4
4   5                                     Pandora's Box           5
5   6                               The Broadway Melody           6
6   7                                     Hell's Angels           7
7   8                                A Farewell to Arms           8
8   9                                       42nd Street           9
9  10                                She Done Him Wrong          10


C:\Users\Hassan\AppData\Local\Temp\ipykernel_3744\253893807.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(query, conn)


### PARTITION BY in Window Functions
`PARTITION BY` divides the data into subsets (partitions) and applies the window function within each subset.

Example: Assigning `ROW_NUMBER()` to films grouped by `release_year`. The numbering restarts for each year.


In [18]:
query = """
SELECT 
    release_year, 
    title, 
    ROW_NUMBER() OVER (PARTITION BY release_year ORDER BY gross DESC) AS row_number
FROM films
WHERE release_year IS NOT NULL
ORDER BY release_year, row_number
LIMIT 20;
"""

try:
    result = pd.read_sql(query, conn)
    print("ROW_NUMBER() with PARTITION BY release_year:")
    print(result)
except Exception as e:
    print(f"Error executing query: {e}")


ROW_NUMBER() with PARTITION BY release_year:
    release_year                                             title  row_number
0           1916  Intolerance: Love's Struggle Throughout the Ages           1
1           1920                    Over the Hill to the Poorhouse           1
2           1925                                    The Big Parade           1
3           1927                                        Metropolis           1
4           1929                               The Broadway Melody           1
5           1929                                     Pandora's Box           2
6           1930                                     Hell's Angels           1
7           1932                                A Farewell to Arms           1
8           1933                                She Done Him Wrong           1
9           1933                                       42nd Street           2
10          1934                             It Happened One Night           1
11     

C:\Users\Hassan\AppData\Local\Temp\ipykernel_3744\938330761.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(query, conn)


#### ORDER BY in Window Functions
Adding `ORDER BY` to the `OVER` clause specifies how rows should be sorted within the partition before the window function is applied.

Example: Ranking films by `gross` within each `release_year`.


In [19]:
query = """
SELECT 
    release_year, 
    title, 
    gross, 
    RANK() OVER (PARTITION BY release_year ORDER BY gross DESC) AS rank
FROM films
WHERE release_year IS NOT NULL AND gross IS NOT NULL
ORDER BY release_year, rank
LIMIT 20;
"""

try:
    result = pd.read_sql(query, conn)
    print("RANK() with PARTITION BY release_year and ORDER BY gross:")
    print(result)
except Exception as e:
    print(f"Error executing query: {e}")


RANK() with PARTITION BY release_year and ORDER BY gross:
    release_year                            title      gross  rank
0           1920   Over the Hill to the Poorhouse    3000000     1
1           1927                       Metropolis      26435     1
2           1929              The Broadway Melody    2808000     1
3           1929                    Pandora's Box       9950     2
4           1933                      42nd Street    2300000     1
5           1935                          Top Hat    3000000     1
6           1936                     Modern Times     163245     1
7           1937  Snow White and the Seven Dwarfs  184925485     1
8           1939               Gone with the Wind  198655278     1
9           1939                 The Wizard of Oz   22202612     2
10          1940                        Pinocchio   84300000     1
11          1940                         Fantasia   76400000     2
12          1942                            Bambi  102797150     1
13  

C:\Users\Hassan\AppData\Local\Temp\ipykernel_3744\3135609524.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(query, conn)


### Combining Multiple Window Functions
We can calculate multiple window functions in a single query. For example:
- `ROW_NUMBER()` assigns a unique number to each row in the partition.
- `RANK()` gives the rank of a row within its partition, with gaps for tied values.
- `DENSE_RANK()` is like `RANK()`, but without gaps.

Example: Applying these functions to rank actors (`people`) based on their number of roles.


In [20]:
query = """
SELECT 
    p.name, 
    COUNT(r.id) AS total_roles,
    ROW_NUMBER() OVER (ORDER BY COUNT(r.id) DESC) AS row_number,
    RANK() OVER (ORDER BY COUNT(r.id) DESC) AS rank,
    DENSE_RANK() OVER (ORDER BY COUNT(r.id) DESC) AS dense_rank
FROM people p
LEFT JOIN roles r ON p.id = r.person_id
GROUP BY p.id, p.name
ORDER BY rank
LIMIT 10;
"""

try:
    result = pd.read_sql(query, conn)
    print("ROW_NUMBER(), RANK(), and DENSE_RANK() for actors based on total roles:")
    print(result)
except Exception as e:
    print(f"Error executing query: {e}")


ROW_NUMBER(), RANK(), and DENSE_RANK() for actors based on total roles:
             name  total_roles  row_number  rank  dense_rank
0  Robert De Niro           54           1     1           1
1  Morgan Freeman           47           2     2           2
2     Johnny Depp           41           3     3           3
3    Bruce Willis           40           4     4           4
4   Steve Buscemi           39           5     5           5
5      Matt Damon           38           6     6           6
6  Clint Eastwood           36           7     7           7
7     Liam Neeson           34          10     8           8
8     Bill Murray           34           9     8           8
9       Brad Pitt           34           8     8           8


C:\Users\Hassan\AppData\Local\Temp\ipykernel_3744\2392903805.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(query, conn)


### Window Functions for Aggregate Calculations
Aggregate functions can also use `OVER` to compute running totals or averages without grouping rows into a single output.

Example: Calculating cumulative gross revenue for films sorted by `release_year`.


In [21]:
query = """
SELECT 
    release_year, 
    title, 
    gross,
    SUM(gross) OVER (PARTITION BY release_year ORDER BY gross DESC) AS cumulative_gross
FROM films
WHERE release_year IS NOT NULL AND gross IS NOT NULL
ORDER BY release_year, cumulative_gross DESC
LIMIT 20;
"""

try:
    result = pd.read_sql(query, conn)
    print("Cumulative gross revenue for films:")
    print(result)
except Exception as e:
    print(f"Error executing query: {e}")


Cumulative gross revenue for films:
    release_year                            title      gross  cumulative_gross
0           1920   Over the Hill to the Poorhouse    3000000         3000000.0
1           1927                       Metropolis      26435           26435.0
2           1929                    Pandora's Box       9950         2817950.0
3           1929              The Broadway Melody    2808000         2808000.0
4           1933                      42nd Street    2300000         2300000.0
5           1935                          Top Hat    3000000         3000000.0
6           1936                     Modern Times     163245          163245.0
7           1937  Snow White and the Seven Dwarfs  184925485       184925485.0
8           1939                 The Wizard of Oz   22202612       220857890.0
9           1939               Gone with the Wind  198655278       198655278.0
10          1940                         Fantasia   76400000       160700000.0
11          1940

C:\Users\Hassan\AppData\Local\Temp\ipykernel_3744\3420081276.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(query, conn)


### Frame Specification in Window Functions
In window functions, the `ROWS` or `RANGE` clause defines the "frame" of rows that the function operates over within the window. This frame can be dynamically defined using keywords like:
- **UNBOUNDED PRECEDING**: Includes all rows from the start of the partition up to the current row.
- **PRECEDING**: Includes a specified number of rows before the current row.
- **CURRENT ROW**: Includes only the current row.
- **FOLLOWING**: Includes a specified number of rows after the current row.
- **UNBOUNDED FOLLOWING**: Includes all rows from the current row to the end of the partition.

**Default Behavior:** If no frame is specified, most functions implicitly use `RANGE UNBOUNDED PRECEDING AND CURRENT ROW`.

#### Example 1: Cumulative Sum with UNBOUNDED PRECEDING
We calculate the running total gross revenue for films within each `release_year`.


In [22]:
query = """
SELECT 
    release_year, 
    title, 
    gross,
    SUM(gross) OVER (PARTITION BY release_year ORDER BY gross DESC ROWS UNBOUNDED PRECEDING) AS cumulative_gross
FROM films
WHERE release_year IS NOT NULL AND gross IS NOT NULL
ORDER BY release_year, cumulative_gross DESC
LIMIT 20;
"""

try:
    result = pd.read_sql(query, conn)
    print("Cumulative gross revenue (UNBOUNDED PRECEDING):")
    print(result)
except Exception as e:
    print(f"Error executing query: {e}")


Cumulative gross revenue (UNBOUNDED PRECEDING):
    release_year                            title      gross  cumulative_gross
0           1920   Over the Hill to the Poorhouse    3000000         3000000.0
1           1927                       Metropolis      26435           26435.0
2           1929                    Pandora's Box       9950         2817950.0
3           1929              The Broadway Melody    2808000         2808000.0
4           1933                      42nd Street    2300000         2300000.0
5           1935                          Top Hat    3000000         3000000.0
6           1936                     Modern Times     163245          163245.0
7           1937  Snow White and the Seven Dwarfs  184925485       184925485.0
8           1939                 The Wizard of Oz   22202612       220857890.0
9           1939               Gone with the Wind  198655278       198655278.0
10          1940                         Fantasia   76400000       160700000.0
11  

C:\Users\Hassan\AppData\Local\Temp\ipykernel_3744\297529719.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(query, conn)


#### Example 2: Rolling Average with PRECEDING and FOLLOWING
We calculate the rolling average IMDb score for reviews. The frame includes the current row, the two preceding rows, and the two following rows.


In [25]:
query = """
SELECT 
    film_id, 
    imdb_score, 
    AVG(imdb_score) OVER (PARTITION BY film_id ORDER BY imdb_score ROWS BETWEEN 2 PRECEDING AND 2 FOLLOWING) AS rolling_avg
FROM reviews
WHERE imdb_score IS NOT NULL
ORDER BY film_id, imdb_score
LIMIT 20;
"""

try:
    result = pd.read_sql(query, conn)
    print("Rolling average IMDb score (2 PRECEDING, 2 FOLLOWING):")
    print(result)
except Exception as e:
    print(f"Error executing query: {e}")


Rolling average IMDb score (2 PRECEDING, 2 FOLLOWING):
    film_id  imdb_score  rolling_avg
0         1         8.0          8.0
1         2         4.8          4.8
2         3         8.3          8.3
3         4         8.3          8.3
4         5         8.0          8.0
5         6         6.3          6.3
6         7         7.8          7.8
7         8         6.6          6.6
8         9         7.7          7.7
9        10         6.5          6.5
10       11         8.2          8.2
11       12         7.8          7.8
12       13         8.6          8.6
13       14         7.1          7.1
14       15         7.7          7.7
15       16         7.8          7.8
16       17         7.0          7.0
17       18         8.0          8.0
18       19         8.2          8.2
19       20         8.2          8.2


C:\Users\Hassan\AppData\Local\Temp\ipykernel_3744\2936787995.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(query, conn)


#### Example 3: Using UNBOUNDED FOLLOWING
We calculate the remaining gross for films, which includes the sum of all gross revenues from the current row to the end of the partition.


In [24]:
query = """
SELECT 
    release_year, 
    title, 
    gross,
    SUM(gross) OVER (PARTITION BY release_year ORDER BY gross DESC ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING) AS remaining_gross
FROM films
WHERE release_year IS NOT NULL AND gross IS NOT NULL
ORDER BY release_year, remaining_gross DESC
LIMIT 20;
"""

try:
    result = pd.read_sql(query, conn)
    print("Remaining gross revenue (UNBOUNDED FOLLOWING):")
    print(result)
except Exception as e:
    print(f"Error executing query: {e}")


Remaining gross revenue (UNBOUNDED FOLLOWING):
    release_year                            title      gross  remaining_gross
0           1920   Over the Hill to the Poorhouse    3000000        3000000.0
1           1927                       Metropolis      26435          26435.0
2           1929              The Broadway Melody    2808000        2817950.0
3           1929                    Pandora's Box       9950           9950.0
4           1933                      42nd Street    2300000        2300000.0
5           1935                          Top Hat    3000000        3000000.0
6           1936                     Modern Times     163245         163245.0
7           1937  Snow White and the Seven Dwarfs  184925485      184925485.0
8           1939               Gone with the Wind  198655278      220857890.0
9           1939                 The Wizard of Oz   22202612       22202612.0
10          1940                        Pinocchio   84300000      160700000.0
11          1940 

C:\Users\Hassan\AppData\Local\Temp\ipykernel_3744\3799183158.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(query, conn)


### Closing the Database Connection
Always close the database connection when done to free up resources.

In [26]:
conn.close()
print("Database connection closed.")

Database connection closed.
